In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate() 
using RobotZoo:Quadrotor
using RobotDynamics
import ForwardDiff as FD
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

import MeshCat as mc
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

include(joinpath(@__DIR__,"utils/cartpole_animation.jl"))

using Plots
# using Printf

  Activating environment at `~/Documents/REx_Lab/tinympc-julia/codegen_examples/Project.toml`


In [2]:
# Define ground truth cartpole ODE for simulation
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    return [qd;qdd]

end

function rk4(params::NamedTuple, x::Vector,u,dt::Float64)
    k1 = dt*dynamics(params, x, u)
    k2 = dt*dynamics(params, x + k1/2, u)
    k3 = dt*dynamics(params, x + k2/2, u)
    k4 = dt*dynamics(params, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [3]:
# Define problem parameters
params = (mc = 1.2, mp = 0.16, l = 0.55)

# states and control sizes 
nx = 4 
nu = 1 

# desired x and u
xgoal = [0, pi, 0, 0]
ugoal = [0]

# initial condition
x0 = xgoal + [10, 0, 0, 0]

# simulation size 
dt = 0.01
tf = 10.0 
t_vec = 0:dt:tf
Nsim = length(t_vec)+100
Nh = 10 # horizon length

# define cost matrices
Q = diagm([1,1,0.05,1])
Qf = 10*Q
R = [0.1]
R = reshape(R,(nu,nu))
ρ = 0.1

# Determine linearized dynamics of the cartpole about its upright position
A = FD.jacobian(dx -> rk4(params, dx, ugoal, dt), xgoal)
B = FD.jacobian(du -> rk4(params, xgoal, du, dt), ugoal)

xmin = -5 .* ones((nx,Nh))
xmax = 5 .* ones((nx,Nh))
umin = -5 .* ones((nu,Nh))
umax = 5 .* ones((nu,Nh))

# Solver options
abs_pri_tol = 1e-3
rel_pri_tol = 1e-3
max_iter = 100
verbose = 1

tinympc_dir = "/Users/anoushkaalavill/Documents/REx_Lab/TinyMPC/"
output_dir = "/generated_code"

"/generated_code"

In [4]:
# Create wrappers for the C backend
using Libdl
# create pointer to shared library
lib = dlopen("/Users/anoushkaalavill/Documents/REx_Lab/TinyMPC/generated_code/build/src/libtiny_wrapper.dylib")

# create pointers to tiny_wrapper functions
set_x = dlsym(lib, "set_x")
function ccall_set_x!(x0)
    @ccall $set_x(x0::Ptr{Cfloat})::Cvoid 
end

set_xref = dlsym(lib, "set_xref")
function ccall_set_xref!(xref)
    @ccall $set_xref(xref::Ptr{Cfloat})::Cvoid 
end

reset_dual_variables = dlsym(lib, "reset_dual_variables")
function ccall_reset_dual_variables!()
    @ccall $reset_dual_variables()::Cvoid
end

call_tiny_solve = dlsym(lib, "call_tiny_solve")
function ccall_tiny_solve!()
    @ccall $call_tiny_solve()::Cint
end

get_x = dlsym(lib, "get_x")
function ccall_get_x!(x_soln)
    @ccall $get_x(x_soln::Ptr{Cfloat})::Cvoid
end

get_u = dlsym(lib, "get_u_soln")
function ccall_get_u!(u_soln)
    @ccall $get_u(u_soln::Ptr{Cfloat})::Cvoid
end

set_xmin = dlsym(lib, "set_xmin")
function ccall_set_xmin!(xmin)
    @ccall $set_xmin(xmin::Ptr{Cfloat})::Cvoid
end

set_xmax = dlsym(lib, "set_xmax")
function ccall_set_xmax!(xmax)
    @ccall $set_xmax(xmax::Ptr{Cfloat})::Cvoid
end

set_umin = dlsym(lib, "set_umin")
function ccall_set_umin!(umin)
    @ccall $set_umin(umin::Ptr{Cfloat})::Cvoid
end

set_umax = dlsym(lib, "set_umax")
function ccall_set_umax!(umax)
    @ccall $set_umax(umax::Ptr{Cfloat})::Cvoid
end

ccall_set_umax! (generic function with 1 method)

In [5]:
# # tests
# @show x0
# ccall_set_x!(x0)
# ccall_reset_dual_variables!()
# ccall_tiny_solve!()

xgoal = Array{Float32}(xgoal)
ccall_set_xref!(xgoal)

xmin = [-100,-100,-100,-100]
xmax = [100,100,100,100]
xmin = Array{Float32}(xmin)
xmax = Array{Float32}(xmax)

umin = [-100]
umax = [100]
umin = Array{Float32}(umin)
umax = Array{Float32}(umax)


# x_soln = Array{Float32}(x_soln)
# ccall_print_x!(x_soln)
# @show Array{Float32}(x_soln)

1-element Vector{Float32}:
 100.0

In [6]:
# MPC loop

x_hist = [Array{Float32}(zeros(nx)) for i = 1:Nsim] # store x values for sim
Nmax = 1
for k=1:Nsim
    # set initial condition and enforce float type
    x0 = vec(x0)
    x0 = Array{Float32}(x0)
    x_hist[k] = 1*x0
    ccall_set_x!(x0)
    x0 = Array{Float32}(x0)

    ccall_set_xref!(xgoal)

    ccall_set_xmin!(xmin)
    ccall_set_xmax!(xmax)

    ccall_set_umin!(umin)
    ccall_set_umax!(umax)

    # reset dual variables
    ccall_reset_dual_variables!()

    # solve
    exitflag = ccall_tiny_solve!()

    # get u solution
    u_soln = zeros((nu,Nh-1))
    u_soln = Array{Float32}(u_soln)
    ccall_get_u!(u_soln)
    u_soln = Array{Float32}(u_soln)
    uk = Array{Float32}(u_soln)[1,1]
    
    xk = rk4(params,x0,uk,dt)
    x0 = 1*xk
    @show Array{Float32}(xk)
end

tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = 

Float32[10.000396, 3.142312, 0.079124786, 0.14390603]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.001542, 3.1443973, 0.1500708, 0.27319363]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.003359, 3.1477072, 0.21328062, 0.38889763]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.005771, 3.1521108, 0.26917467, 0.49198765]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.008707, 3.1574867, 0.31815237, 0.58337206]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.012101, 3.163722, 0.36059335, 0.66390234]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.015888, 3.1707122, 0.39685848, 0.7343768]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.020009, 3.1783605, 0.4272911, 0.7955445]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[10.024406, 3.1865773, 0.45221776, 0.8481088]
tiny sol

tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.348467, 3.4718347, -2.671778, -0.35476267]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.321558, 3.4682467, -2.709961, -0.36293843]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.2942705, 3.464579, -2.7475107, -0.3707166]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.266611, 3.4608357, -2.784423, -0.3781024]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.238585, 3.4570203, -2.820695, -0.3851012]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.2102, 3.453137, -2.8563232, -0.39171866]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.181462, 3.4491892, -2.891306, -0.39796057]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[9.152377, 3.445181, -2.9256413, -0.40383294]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Floa


Array{Float32}(xk) = Float32[6.213398, 3.1302297, -3.9270287, -0.27384627]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[6.174144, 3.1275096, -3.9238212, -0.2702846]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[6.134923, 3.124825, -3.92036, -0.26674408]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[6.095738, 3.1221757, -3.916649, -0.26322526]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[6.056591, 3.1195614, -3.9126918, -0.25972858]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[6.017485, 3.116982, -3.908492, -0.2562545]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[5.9784226, 3.114437, -3.9040534, -0.2528034]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[5.9394054, 3.1119266, -3.8993793, -0.24937557]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[5.9004364, 3.1094503, -3.894473, -


Array{Float32}(xk) = Float32[5.0595455, 3.0631151, -3.73443, -0.17734694]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[5.0222483, 3.061356, -3.7250652, -0.17451368]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.9850454, 3.0596254, -3.7155414, -0.17170465]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.9479384, 3.0579226, -3.705861, -0.1689198]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.910929, 3.0562475, -3.696027, -0.16615896]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.8740187, 3.0545998, -3.6860416, -0.16342196]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.8372087, 3.0529795, -3.6759076, -0.16070865]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.800501, 3.051386, -3.6656275, -0.15801895]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[4.763897, 3.0498195, -3.655

...


Array{Float32}(xk) = Float32[-0.14466392, 3.1455414, 0.119955994, 0.003898233]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.14346652, 3.14558, 0.119524345, 0.0038211865]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.14227346, 3.145618, 0.119089305, 0.003744802]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.14108476, 3.145655, 0.118650936, 0.0036690612]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.13990046, 3.1456912, 0.11820937, 0.003594025]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.13872059, 3.1457267, 0.11776466, 0.0035196755]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.13754518, 3.1457615, 0.11731687, 0.0034459978]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.13637426, 3.1457956, 0.11686607, 0.00337298]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32

tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.06672281, 3.1467202, 0.08136418, -0.00027896967]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.06591178, 3.1467173, 0.08084269, -0.00031281993]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.06510595, 3.146714, 0.08032172, -0.00034626617]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.064305335, 3.1467104, 0.07980134, -0.00037926328]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.06350992, 3.1467063, 0.07928156, -0.000411845]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.062719695, 3.146702, 0.07876244, -0.00044396584]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.061934665, 3.1466975, 0.078244, -0.00047565764]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.061154813, 3.1466925, 0.07772624, -0.0005069495]
tiny solve 

tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.016171271, 3.1456795, 0.042107813, -0.0018563517]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.015752211, 3.1456609, 0.041704066, -0.0018631523]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.015337179, 3.1456423, 0.041302267, -0.0018697291]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.014926156, 3.1456234, 0.040902406, -0.0018761061]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.0145191215, 3.1456046, 0.040504538, -0.0018822291]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.014116055, 3.1455858, 0.040108643, -0.0018881235]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.013716938, 3.145567, 0.039714713, -0.0018938119]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[-0.013321751, 3.1455479, 0.039322738, -0.0018993146]



Array{Float32}(xk) = Float32[0.006585431, 3.144091, 0.015630249, -0.0018584372]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.006740526, 3.1440723, 0.01538871, -0.0018536584]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.006893215, 3.1440537, 0.015149043, -0.0018487909]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.0070435163, 3.1440353, 0.014911253, -0.001843822]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.007191449, 3.144017, 0.014675282, -0.0018388152]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.0073370314, 3.1439986, 0.01444114, -0.0018337528]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.0074802814, 3.1439803, 0.0142088365, -0.0018286187]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.0076212175, 3.143962, 0.013978379, -0.0018233977]
tiny solve finished!
exitflag = 1612384480
Array{Fl


Array{Float32}(xk) = Float32[0.012671149, 3.1428974, 0.0024886087, -0.0013786438]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.01269542, 3.1428835, 0.0023654513, -0.0013710797]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.012718465, 3.14287, 0.0022436648, -0.0013634829]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.0127403, 3.1428564, 0.0021231957, -0.0013559081]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.012760936, 3.1428428, 0.002004058, -0.0013483292]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.012780388, 3.1428294, 0.0018862651, -0.0013407229]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.012798668, 3.142816, 0.0017697635, -0.0013331432]
tiny solve finished!
exitflag = 1612384480
Array{Float32}(xk) = Float32[0.012815789, 3.1428027, 0.0016545678, -0.001325564]
tiny solve finished!
exitflag = 1612384480
Array{F

In [7]:
# Simulate
display(animate_cartpole(x_hist[1:Nsim], dt))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /Users/anoushkaalavill/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "b" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x62], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x63, 0x61, 0x72, 0x74], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "a" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x61], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{Any}(), ip"127.0.0.1", 8700))

In [8]:
@show B

B = [4.166749242424243e-5; 7.577033746556473e-5; 0.008333663636363637; 0.015156619834710741]


4×1 Matrix{Float64}:
 4.166749242424243e-5
 7.577033746556473e-5
 0.008333663636363637
 0.015156619834710741